# 一、简介
- 参考：(https://huggingface.co/blog/zh/chat-templates)

## 1. 为什么哟啊使用chat模版？
在使用hugging-face库时，我们加载Tokenizer和model都是使用同一个模型的相关信息，如下所示：
```
# 加载同一个checkpoint的Tokenizer和model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
```
如果Tokenizer和model加载的不是同一个checkpoint，那么最坏的情况下在编码chat prompt时，某些token id不在训练时的token中，相当于ood了。就算在，也有可能id不是同一个，也有分布偏移的问题。

## 2. 不同的chat prompt模版
一般的chat prompt会有role和content。role分别有：
- user：用于发送消息
- assistant：用于模型生成回答
- system：在对话开始时给出高级指令
一般用一个字典表示一条消息（message），多条message组合成一个列表表示一个对话（dialog）
```
[
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Nice to meet you!"}
]
```
那么转换成prompt可能多种多样：
- 即时消息型
```
User: Hey there!
Bot: Nice to meet you!
```
- 添加特殊token指代role
```
[USER] Hey there! [/USER]
[ASST] Nice to meet you! [/ASST]
```
- 添加特殊token切分边界
```
<|im_start|>user
Hey there!<|im_end|>
<|im_start|>assistant
Nice to meet you!<|im_end|>
```

# 二、Chat Templates：一种保存格式信息的方式
- 主要是存格式
- 用的是[jinja模版字符串](https://jinja.palletsprojects.com/en/3.1.x/)

## 1. jinja模版
上述三种chat格式对应下面三种jinja模版：
- 即时消息型
```
{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{ "User : " }}
    {% else %}
        {{ "Bot : " }}
    {{ message['content'] + '\n' }}
{% endfor %}
```
- 添加特殊token指代role
```
{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{ "[USER]" + message['content'] + " [/USER]" }}
    {% else %}
        {{ "[ASST]" + message['content'] + " [/ASST]" }}
    {{ message['content'] + '\n' }}
{% endfor %}
```
- 添加特殊token切分边界
```
"{% for message in messages %}"
    "{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}"
"{% endfor %}"
```

## 2. 为什么使用jinja模版？
- 因为目前各个LLM使用不同的模版训练，而且很难统一起来
- 因为它能够用统一的语言编码我们目前所有使用的chat prompt模版
- hugging-face很好的集成了chat_template相关方法

# 3. 如何使用模版？
可以查看[开发指南](https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.apply_chat_template)

虽然默认提供一套模版，但是最好是显式指定模版：
```
tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

```

另外在开发自己的LLM，最好将jinja的chat模版上传，并提交PR